In [25]:
import time
from functools import wraps
def timethis(func):
    @wraps(func)  # @wraps会复制被装饰器函数元信息给装饰器
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n: int) -> None:
    '''Counts down'''
    while n > 0:
        n -= 1
        
countdown(10**6)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)
print(countdown.__wrapped__) # @wraps能让你通过属性__wrapped__直接访问被装饰函数
from inspect import signature
print(signature(countdown)) # __wrapped__能让被装饰函数正确暴露底层的参数签名信息

countdown 0.0468747615814209
countdown
Counts down
{'n': <class 'int'>, 'return': None}
<function countdown at 0x000001DF551FA3A8>


In [24]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n: int) -> None:
    '''Counts down'''
    while n > 0:
        n -= 1
        
countdown(10**6)
# 没有使用@wraps则被装饰器函数countdown丢失所有元信息
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)

countdown 0.045876502990722656
wrapper
None
{}


In [35]:
from inspect import signature
from functools import wraps

# 利用装饰器强制函数上的类型检查
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
#         print(sig)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
#         print(bound_types)

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

spam(1, 2, 3)

(x, y, z=42)
OrderedDict([('x', <class 'int'>), ('z', <class 'int'>)])
1 2 3


In [45]:
class C:
    def __init__(self):
          self.__x=None
    def getx(self):
          return self.__x
    def setx(self,value):
          self.__x=value
    def delx(self):
          del self.__x
    x=property(getx,setx,delx,'')
c = C()
c.x = 2
c.x
del c.x

In [53]:
class C(object):
    def __init__(self):
        self._x = None

    @property    
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

c = C()
c.x = 3
print(c.x)

3


In [459]:
class C(object):
    def __init__(self):
        self._x = None

    x = property()

    @x.getter
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

c = C()
print(C.x)

In [50]:
from functools import wraps
 
class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile
 
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
 
    def notify(self):
        # logit只打日志，不做别的
        pass

kk = logit() 
print(kk)
def test():
    pass
cc = kk(test)
cc

<function __main__.test()>

In [96]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            print(args[0].a)
            return func(*args, **kwargs)
        return wrapper

class B(A):
    a = 5
    @A.decorator2
    def bar(self):
        return 2
b = B()
print(b.bar())

Decorator 2
5
2


In [116]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.signature(func).parameters:
        raise TypeError('debug argument already defined')

    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    sig = inspect.signature(func)
    print(sig.parameters)
    parms = list(sig.parameters.values())
    print(parms)
    parms.append(inspect.Parameter('debug',
                inspect.Parameter.KEYWORD_ONLY,
                default=False))
    print(parms)
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

@optional_debug
def add(x, y):
    return x+y

print(inspect.signature(add))


OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y">)])
[<Parameter "x">, <Parameter "y">]
[<Parameter "x">, <Parameter "y">, <Parameter "debug=False">]
(x, y, *, debug=False)


In [119]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

a = A(2)
print(a.x)
print(a.spam())

getting: x
2
getting: spam
None


In [125]:
# 创建不能实例化对象的类
class NoInstances(type):
    pass
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

# Example
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [409]:
# 创建单例
class Singleton(type):
    def __init__(self, *args, **kwargs): # 被父元类的__call__方法调用，初始化创建的类对象Spam
        self.__instance = None # 类对象的属性__instance设置为None
        super().__init__(*args, **kwargs)

    def __call__(cls, *args, **kwargs): # cls是类Spam,创建实例对象是，该方法吧隐式被类Spam调用，创建Spam的实例对象
        if cls.__instance is None:
            cls.__instance = super().__call__(*args, **kwargs) # 调用类的超类的__call__方法，返回类的实例对象
            return cls.__instance
        else:
            return cls.__instance

# Example
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

cc = Spam()
cc2 = Spam()
cc == cc2

Creating Spam


True

In [168]:
import weakref

class Cache(object):
    def __init__(self):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary() # 创建一个弱引用值的映射:值删除后,弱引用隐射对应的元素项也会被垃圾回收删除
    
    def __call__(self, *args):
        if args in self.__cache:
            return sefl.__cache[args]
        else:
            obj = super.__call__(*args)
            self.__cache[args] = obj
            return obj
a = Spam('GG')
b = Spam('GG')
a == b # 显示为True

True

In [209]:
from collections import OrderedDict


# A set of descriptors for various types
class Typed:
    '''数据描述器'''
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value): # 访问描述器对象设置属性时，隐式调用该方法_
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value # 设置实例的__dict__命名空间

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    ''' 捕获类的定义信息'''
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict) # clsdict即空间钩子__prepare__返回的OrderedDict()，也是类的namespace
        order = []
        for name, value in clsdict.items(): # 找出类属性中所有Typed的实例组成dict，放入类的'_order’属性中
            if isinstance(value, Typed):
                value._name = name # 修改类型对象的_name属性值为属性名，实例属性名和类属性名相同时，只会访问实例属性，类的属性会被隐藏
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases): # 创建namespace:类属性字典clsdict，返回类型为OrderdDict
        return OrderedDict()

class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

# Example use
class Stock(Structure):
    name = String() # 创建规定的类对象，实例化时必须按按照类型顺序传入构造函数__init__
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price): # 要求传入数据的类型顺序为String、Interger、Float，否者属性访问时就会抛出错误
        self.name = name
        self.shares = shares
        self.price = price
s = Stock('good', 100, 490.1)
s.as_csv()

'good,100,490.1'

In [222]:
from collections import OrderedDict

class NoDupOrderedDict(OrderedDict):
    '''自己构造的类字典对象，重复定义则抛出异常'''
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()
        
    def __setitem__(self, name, value): # 隐式设置dict时(即将要创建的类的类属性放入该dict种)，name()已key经定义则抛出异常
        if name in self:  
            raise TypeError(f'{name} already defined in {self.clsname}')
        super().__setitem__(name, value)

class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        print(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_'] # 取出非'_'开头的key组成一个字典元素
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases): # 在__new__方法之前执行，创建namespace，就是字典类对象
        return NoDupOrderedDict(clsname)

class A(metaclass=OrderedMeta):
    def test(self):
        pass
    def test2(self):
        pass

NoDupOrderedDict([('__module__', '__main__'), ('__qualname__', 'A'), ('test', <function A.test at 0x000001DF55727168>), ('test2', <function A.test2 at 0x000001DF55727798>)])


In [286]:
from inspect import signature, Signature, Parameter

def func(x, y=42, *, z=None):
    pass

signature(func) # 创建函数签名:<Signature (x, y=42, *, z=None)>

<Signature (x, y=42, *, z=None)>

In [307]:
from inspect import Signature, Parameter

parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
         Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]

sig = Signature(parms) #  创建函数签名:<Signature (x, y=42, *, z=None)>

def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name, value)

func(1, 2, z=3)

x 1
y 2
z 3


In [310]:
from inspect import signature, Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
            for name in names]
    return Signature(parms)

class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[])) # 使用类属性_field创建类的签名属性__signature__
        return super().__new__(cls, clsname, bases, clsdict)

class Structure(metaclass=StructureMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs) # 强制签名检查
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Example
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']
    
print(signature(Stock)) # 将签名存储在特定的属性 __signature__中，在inspect模块执行内省的代码就能发现签名并将它作为调用约定

(name, shares, price)


In [312]:
from inspect import signature
import logging

class MatchSignaturesMeta(type):

    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)
        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue
            # Get the previous definition (if any) and compare the signatures
            prev_dfn = getattr(sup,name,None) # 获取父类被重载的方法
            if prev_dfn:
                prev_sig = signature(prev_dfn) # 获取父类的被重载方法的参数签名
                val_sig = signature(value) # 获取子类重载方法的参数签名
                if prev_sig != val_sig: # 参数签名不等及抛出警告信息
                    logging.warning('Signature mismatch in %s. %s != %s',
                                    value.__qualname__, prev_sig, val_sig)

# Example
class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass

    def spam(self, x, *, z):
        pass

# Class with redefined methods, but slightly different signatures
class B(A):
    def foo(self, a, b):
        pass

    def spam(self,x,z):
        pass

In [336]:

# stock.py
# Example of making a class manually from parts

# Methods
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price
def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}
class Base:
    def __init_subclass__(cls, **kwargs):
        pass

# Make a class
import types
import abc

Stock = types.new_class('Stock', (Base,), {'metaclass': abc.ABCMeta, 'debug': True}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__
d = Stock('ACME', 50, 91.1)
print(d)

In [339]:
import collections

Stock = collections.namedtuple('Stock', ['name', 'shares', 'price'])
stock = Stock('cc', 100, 99)
print(Stock)
print(stock)
print(stock.name)
print(stock[0])

<class '__main__.Stock'>
Stock(name='cc', shares=100, price=99)
cc
cc


In [378]:
import operator
import types
import sys

def named_tuple(classname, fieldnames):
    # 使用property对象填充字典，访问name就是以属性的方式来访问operator.itemgetter(n)
    cls_dict = { name: property(operator.itemgetter(n))
                for n, name in enumerate(fieldnames) }

    # Make a __new__ function and add to the class dict
    def __new__(cls, *args):
        print(args)
        if len(args) != len(fieldnames):
            raise TypeError('Expected {} arguments'.format(len(fieldnames)))
        return tuple.__new__(cls, args)
    
    def __init__(self, *args):
        print(args)

    cls_dict['__new__'] = __new__
    cls_dict['__init__'] = __init__
    # Make the class
    cls = types.new_class(classname, (tuple,), {},
                        lambda ns: ns.update(cls_dict))

    # Set the module to that of the caller
    cls.__module__ = sys._getframe(1).f_globals['__name__']
    return cls

Point = named_tuple('Point', ['x', 'y'])
print(Point)
print(Point(4, 5).x)

<class '__main__.Point'>
(4, 5)
(4, 5)
4


In [406]:
class NameTuple(tuple):
    def __new__(cls, *args):
        return super().__new__(cls, args)
    
    def __init__(self, *args):
        self.x = args[0]
        self.y = args[1]
cc = NameTuple(4, 5)   
print(cc)
print(cc.x)

(4, 5)
4


In [433]:
import types
metaclass, kwargs, ns = types.prepare_class('Stock', (), {'metaclass': type})
print(metaclass, kwargs, ns)

<class 'type'> {} {}


In [453]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
#             setattr(cls, name, property(operator.itemgetter(n))) 
            setattr(cls, name, operator.itemgetter(n)) 

class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    def __new__(cls, *args):
        if len(args) != len(cls._fields):
            raise ValueError('{} arguments required'.format(len(cls._fields)))
        return super().__new__(cls,args)

class Stock(StructTuple):
    _fields = ['name', 'shares', 'price']

class Point(StructTuple):
    _fields = ['x', 'y']

s = Stock('ACME', 50, 91.1)
print(s.name) # ('ACME', 50, 91.1)
print(operator.itemgetter(0)(s))

operator.itemgetter(0)
ACME


In [469]:
def bar(x:int, y:int):
        print('Bar 1:', x, y)

import inspect

print(inspect.signature(bar).parameters['x'].annotation)


<class 'int'>


In [533]:
# 利用函数注解实现方法重载

# multiple.py
import inspect
import types

class MultiMethod:
    '''
    Represents a single multimethod.
    '''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name

    def register(self, meth):
        '''
        注册一个方法为重载方法，核心思想：将参数注解和方法建立映射关系，方法同名时，可根据参数注解找到对应的方法
        '''
        sig = inspect.signature(meth)

        # Build a type signature from the method's annotations
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError(
                    'Argument {} must be annotated with a type'.format(name)
                )
            if not isinstance(parm.annotation, type):
                raise TypeError(
                    'Argument {} annotation must be a type'.format(name)
                )
            if parm.default is not inspect.Parameter.empty: # 有默认值则应先创建一个映射关系，因为默认值参数允许不传值
                self._methods[tuple(types)] = meth
            types.append(parm.annotation) # 存储方法的所有参数注解

        self._methods[tuple(types)] = meth # 将参数注解和方法映射，利用参数注解就可找到方法

    def __call__(self, *args):
        '''
        Call a method based on type signature of the arguments
        '''
        types = tuple(type(arg) for arg in args[1:]) # 去除被绑定的instance对象,获取参数的类型，查找对应重载方法
        meth = self._methods.get(types, None)
        print(self)
        print(args)
        if meth:
            return meth(*args)
        else:
            raise TypeError('No matching method for types {}'.format(types))

    def __get__(self, instance, cls):
        '''
        Descriptor method needed to make calls work in a class
        '''
        if instance is not None:
            # 将实例对象instance(这里指Spam的实例对象)绑定到callable的MultiMethod对象上:将instance作为MultiMethod对象的第一个参数
            return types.MethodType(self, instance) 
        else:
            return self

class MultiDict(dict):
    '''
    Special dictionary to build multimethods in a metaclass
    '''
    def __setitem__(self, key, value): # key\value分别为要创建类的属性名和值(值对于方法来说就是函数对象)
        if key in self:
            # If key already exists, it must be a multimethod or callable
            current_value = self[key]
            if isinstance(current_value, MultiMethod): #该key对应的方法已创建MultiMethod对象
                current_value.register(value) # 注册重载方法
            else:
                mvalue = MultiMethod(key)  # 出现第二个同名方法时，创建MultiMethod对象
                mvalue.register(current_value) # 注册重载方法
                mvalue.register(value) # 注册重载方法
                super().__setitem__(key, mvalue) # 将重载对象添加到命名空间
        else:
            super().__setitem__(key, value) # 第一个方法直接添加到命名空间

class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))

    @classmethod
    def __prepare__(cls, clsname, bases): 
        return MultiDict() # 类的命名空间使用自定义字典111111111111111111111111，隐式以"MultiDict[类的属性key] = 类的属性值"方式填充

# 应用
class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)

    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)

s = Spam()
s.bar(2, 3) # bar是描述器，调用它的__get__方法
s.bar('hello')

(<__main__.Spam object at 0x000001DF56E86308>, 2, 3)
Bar 1: 2 3


In [542]:
class A:
    def __call__(self, *args):
        print(*args)

class B:
    pass

a = A()

import types
b.test = types.MethodType(a, b)
b.test(5)

<__main__.Spam object at 0x000001DF564CDEC8> 5
None
